In [1]:
import sys
import os
import pandas as pd
from pandas.api.types import is_string_dtype
import math
import re
from tqdm import tqdm_notebook, tqdm
import j_utils.munging as mg
from lendingclub import config

rli = pd.read_feather(os.path.join(config.data_dir, 'raw_loan_info.fth'))
# Lendingclub still has internal grades through G
# take a random sample of 100000 loans to work with
# rli = rli.sample(5000, random_state=42)
dev_ids = rli['id']

In [2]:
# for g, group in rli.groupby('grade'):
#     print(pd.to_datetime(group['issue_d']).max())

In [3]:
# print(dev_ids.to_list())

In [4]:
import pickle

with open(os.path.join(config.data_dir,'dev_ids.pkl'), 'wb') as f:
    pickle.dump(dev_ids.to_list(), f)